### Install package

In [ ]:
!pip install pytorch-adapt[ignite]

### Import packages

In [ ]:
import torch

from pytorch_adapt.adapters import DANN
from pytorch_adapt.containers import Models, Optimizers
from pytorch_adapt.datasets import DataloaderCreator, get_mnist_mnistm
from pytorch_adapt.frameworks.ignite import Ignite
from pytorch_adapt.models import Discriminator, mnistC, mnistG
from pytorch_adapt.validators import EntropyDiversityValidator

### Create datasets and dataloaders

In [ ]:
datasets = get_mnist_mnistm(["mnist"], ["mnistm"], folder=".", download=True)
dc = DataloaderCreator(batch_size=32, num_workers=2)

### Create models, optimizers, hook, and validator

In [ ]:
G = mnistG(pretrained=True)
C = mnistC(pretrained=True)
D = Discriminator(in_size=1200, h=256)
models = Models({"G": G, "C": C, "D": D})
optimizers = Optimizers((torch.optim.Adam, {"lr": 0.0001}))

adapter = DANN(models=models, optimizers=optimizers)
wrapped_adapter = Ignite(adapter)
validator = EntropyDiversityValidator()

### Train and evaluate

In [ ]:
wrapped_adapter.run(datasets, dataloader_creator=dc, validator=validator, max_epochs=2)